In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)


In [ ]:
# ! wget "https://zenodo.org/record/1489920/files/articles-training-byarticle-20181122.zip"
# ! unzip "articles-training-byarticle-20181122.zip"
! wget "https://zenodo.org/record/1489920/files/articles-training-bypublisher-20181122.zip"
! unzip "articles-training-bypublisher-20181122.zip"
! wget "https://zenodo.org/record/1489920/files/articles-validation-bypublisher-20181122.zip"
! unzip "articles-validation-bypublisher-20181122.zip" 

--2020-11-16 20:37:34--  https://zenodo.org/record/1489920/files/articles-training-bypublisher-20181122.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980769009 (935M) [application/octet-stream]
Saving to: ‘articles-training-bypublisher-20181122.zip’

articles-training-b 100%[===================>] 935.33M  11.3MB/s    in 95s     

2020-11-16 20:39:10 (9.83 MB/s) - ‘articles-training-bypublisher-20181122.zip’ saved [980769009/980769009]

Archive:  articles-training-bypublisher-20181122.zip
  inflating: articles-training-bypublisher-20181122.xml  
--2020-11-16 20:39:47--  https://zenodo.org/record/1489920/files/articles-validation-bypublisher-20181122.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337386275 (322M) [applicati

In [ ]:
# ! wget "https://zenodo.org/record/1489920/files/ground-truth-training-byarticle-20181122.zip"
# ! unzip "ground-truth-training-byarticle-20181122.zip"
! wget "https://zenodo.org/record/1489920/files/ground-truth-training-bypublisher-20181122.zip"
! unzip "ground-truth-training-bypublisher-20181122.zip"
! wget "https://zenodo.org/record/1489920/files/ground-truth-validation-bypublisher-20181122.zip"
! unzip "ground-truth-validation-bypublisher-20181122.zip"

--2020-11-16 20:40:25--  https://zenodo.org/record/1489920/files/ground-truth-training-bypublisher-20181122.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22426411 (21M) [application/octet-stream]
Saving to: ‘ground-truth-training-bypublisher-20181122.zip’

ground-truth-traini 100%[===================>]  21.39M  16.5MB/s    in 1.3s    

2020-11-16 20:40:27 (16.5 MB/s) - ‘ground-truth-training-bypublisher-20181122.zip’ saved [22426411/22426411]

Archive:  ground-truth-training-bypublisher-20181122.zip
  inflating: ground-truth-training-bypublisher-20181122.xml  
--2020-11-16 20:40:28--  https://zenodo.org/record/1489920/files/ground-truth-validation-bypublisher-20181122.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5239324

In [ ]:
import xml.sax

training_articles = {}
training_ground_dict ={}

In [ ]:
class DataParserTraining(xml.sax.ContentHandler):

    def __init__(self):
        self.title = ""
        self.text = ""
        self.id = ""
        self.date = ""

    def startElement(self, tag, attributes):
        if tag == "article":
            self.text = ""
            self.date = ""
            self.title = ""
            self.title = attributes["title"]
            self.id = attributes["id"]
            if "published-at" in attributes:
                self.date = attributes["published-at"]

    def characters(self, content):

        self.text += content

    def endElement(self, tag):
        if tag == "article":
            lis = [self.date, self.title, self.text]
            training_articles[self.id] = lis  

In [ ]:
class DataParserGroundTruthTrain(xml.sax.ContentHandler):
    def __init__(self):
        self.d = {}
        self.id = ""

    def startElement(self, tag, attributes):
        atts = ["hyperpartisan", "labeled-by", "url", "bias"]
        if tag == "article":
            self.d = {}
            self.id = ""
            self.id = attributes["id"]
            for a in atts:
                if a in attributes:
                    self.d[a] = attributes[a]
        if self.id != "":
            training_ground_dict[self.id] = self.d


In [ ]:
train_parser = xml.sax.make_parser()
handler = DataParserTraining()
train_parser.setContentHandler(handler)
# train_parser.parse('articles-training-byarticle-20181122.xml')
train_parser.parse('articles-training-bypublisher-20181122.xml')
# print(training_articles["0000007"])

In [ ]:
TrainParser = xml.sax.make_parser()
handler = DataParserGroundTruthTrain()
TrainParser.setContentHandler(handler)
# TrainParser.parse('ground-truth-training-byarticle-20181122.xml')
TrainParser.parse('ground-truth-training-bypublisher-20181122.xml')

In [ ]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm
from collections import Counter
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score
from torchtext import data
import json
from torchtext import data

tqdm.pandas()

In [ ]:
train_dataset = []
train_keys = list(training_articles.keys())


In [ ]:
for k in train_keys:
  d = {}
  d['id'] = k
  d['title'] = training_articles[k][1]
  d['text'] = training_articles[k][2]
  d['hyperpartisan'] = training_ground_dict[k]['hyperpartisan']
  d['url'] = training_ground_dict[k]['url']
  d['labeled_by'] = training_ground_dict[k]['labeled-by']
  train_dataset.append(d)

In [ ]:
del train_keys
del training_ground_dict

In [ ]:
json_object= json.dumps(train_dataset,indent=4)
with open("train_article.json", "w") as outfile: 
    outfile.write(json_object)

In [ ]:

%reset -f
import json
import xml.sax
from IPython.display import HTML, display


In [ ]:
validation_ground_dict={}
validation_articles ={}


In [ ]:
class DataParserValidation(xml.sax.ContentHandler):

    def __init__(self):
        self.title = ""
        self.text = ""
        self.id = ""
        self.date = ""

    def startElement(self, tag, attributes):
        if tag == "article":
            self.text = ""
            self.date = ""
            self.title = ""
            self.title = attributes["title"]
            self.id = attributes["id"]
            if "published-at" in attributes:
                self.date = attributes["published-at"]

    def characters(self, content):

        self.text += content

    def endElement(self, tag):
        if tag == "article":
            lis = [self.date, self.title, self.text]
            validation_articles[self.id] = lis  

In [ ]:
class DataParserGroundTruthVal(xml.sax.ContentHandler):
    def __init__(self):
        self.d = {}
        self.id = ""

    def startElement(self, tag, attributes):
        atts = ["hyperpartisan", "labeled-by", "url", "bias"]
        if tag == "article":
            self.d = {}
            self.id = ""
            self.id = attributes["id"]
            for a in atts:
                if a in attributes:
                    self.d[a] = attributes[a]
        if self.id != "":
            validation_ground_dict[self.id] = self.d


In [ ]:
validation_parser = xml.sax.make_parser()
handler = DataParserValidation()
validation_parser.setContentHandler(handler)
validation_parser.parse('articles-validation-bypublisher-20181122.xml')


In [ ]:
ValParser = xml.sax.make_parser()
handler = DataParserGroundTruthVal()
ValParser.setContentHandler(handler)
ValParser.parse('ground-truth-validation-bypublisher-20181122.xml')

In [ ]:
validation_dataset=[]
val_keys = list(validation_articles.keys())

In [ ]:
for k in val_keys:
  d = {}
  d['id'] = k
  d['title'] = validation_articles[k][1]
  d['text'] = validation_articles[k][2]
  d['hyperpartisan'] = validation_ground_dict[k]['hyperpartisan']
  d['url'] = validation_ground_dict[k]['url']
  d['labeled_by'] = validation_ground_dict[k]['labeled-by']
  validation_dataset.append(d)

In [ ]:
json_object= json.dumps(validation_dataset,indent=4)
with open("validation_article.json", "w") as outfile: 
    outfile.write(json_object)

In [ ]:
%reset -f
import json
import xml.sax
from IPython.display import HTML, display


In [ ]:
f = open('train_article.json')
train_df_articles = json.load(f)
for key in train_df_articles :
  if key['hyperpartisan'] == 'true':
    key['hyperpartisan'] = 1
  else :
    key['hyperpartisan'] = 0


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
import string
stop_words = set(stopwords.words('english'))
def process_text(text):
  text = text.lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = re.sub('[ ]+', ' ', text)
  text = text.strip()
  word_tokens = re.split(r'[^A-Za-z0-9]+', text)
  filtered_text = [w for w in word_tokens if w not in stop_words]
  text = (' ').join(filtered_text)
  return text

In [ ]:
for key in train_df_articles :
   processed = process_text(key['text']);
   key['text'] = processed

In [ ]:
with open("train_processed_article.json", "w") as outfile: 
    outfile.write(json.dumps(train_df_articles,indent=4))

In [ ]:
%reset -f
from IPython.display import HTML, display
import json


In [ ]:
f = open('validation_article.json')
validation_articles = json.load(f)
for key in validation_articles :
  if key['hyperpartisan'] == 'true':
    key['hyperpartisan'] = 1
  else :
    key['hyperpartisan'] = 0

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re
import string
stop_words = set(stopwords.words('english'))
def process_text(text):
  text = text.lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = re.sub('[ ]+', ' ', text)
  text = text.strip()
  word_tokens = re.split(r'[^A-Za-z0-9]+', text)
  filtered_text = [w for w in word_tokens if w not in stop_words]
  text = (' ').join(filtered_text)
  return text

In [ ]:
for key in validation_articles :
   processed = process_text(key['text']);
   key['text'] = processed

In [ ]:
with open("validation_processed_article.json", "w") as outfile: 
    outfile.write(json.dumps(validation_articles,indent=4))

In [ ]:
%reset -f
from IPython.display import HTML, display
import json

In [ ]:
!cp train_processed_article.json /content/drive/MyDrive/IRE_project/
# !cp validation_processed_article.json /content/drive/My\ Drive/IRE_project/

cp: cannot stat 'train_processed_article.json': No such file or directory


In [ ]:
!du -csh *.json

2.5G	train_article.json
1.7G	train_processed_article.json
798M	validation_article.json
548M	validation_processed_article.json
5.5G	total


In [ ]:
dict1 = {}
f = open('train_processed_article.json')
train = json.load(f)
for key in train:
  text=key['text'].split()
  for w in text: 
    if w in dict1 :
      dict1[w]+=1;
    else:
      dict1[w] =1;
  

In [ ]:
f = open('/content/drive/My Drive/dict1.json')
dict1 = json.load(f)

In [ ]:
f = open('validation_processed_article.json')
val= json.load(f)
# for key in val:
#   text=key['text'].split()
#   for w in text: 
#     if w in dict1 :
#       dict1[w]+=1;
#     else:
#       dict1[w] =1;
  

In [ ]:
print(len(dict1))
json_data = json.dumps(dict1)
with open('dict1.json', 'w') as out:
  out.write(json_data)

1995844


In [ ]:
# f = open('dict1.json')
# dict1 = json.load(f)
dictsort = sorted(dict1, key=dict1.get, reverse=True)

In [ ]:
for i in range(0,10):
  t = dictsort[i];
  print(t)
  print(dict1[t])

said
2021033
us
1316630
new
1250519
one
1178078
would
1161995
people
1002072
also
843455
state
732414
like
703121
president
665066


In [ ]:
for i in range(len(dict1)):
  t = dictsort[i];
  dict1[t]= i+1;

In [ ]:
for key in train:
  text=key['text'].split()
  listtext=[]
  for w in text:
    if w in dict1:
      listtext.append(dict1[w]);
    else:
      listtext.append(0)
  key['text'] = listtext


In [ ]:
from torchtext import data
import numpy as np
import json

In [ ]:
def pad_features(reviews_int, seq_length):
    features = np.zeros((seq_length), dtype=np.int32)
    # print(len(reviews_int))
    # print(features.shape)
    # for i, row in enumerate(reviews_int):
    #     features[i, -len(row):] = np.array(row)[:seq_length]
    features[-len(reviews_int):] = np.array(reviews_int)[:seq_length]
    return features

In [ ]:
for key in train:
  key['text'] = pad_features(key['text'],512)

In [ ]:
def convert_type(item):
  item = np.array(item, dtype=np.int32)
  return item

In [ ]:
for key in train:
  key['text'] = convert_type(key['text'])

In [ ]:
for key in val:
  text=key['text'].split()
  listtext=[]
  for w in text: 
    listtext.append(dict1[w]);
  key['text'] = listtext


In [ ]:
import numpy as np
for key in val:
  key['text'] = pad_features(key['text'],512)

In [ ]:
for key in val:
  key['text'] = convert_type(key['text'])

In [ ]:
train_features = []
train_labels=[]
for item in train:
  train_features.append(np.array(item['text']))
  train_labels.append(item['hyperpartisan'])
train_features = np.array(train_features)
len(train_features)

645

In [ ]:
val_features = []
val_labels=[]
for item in val:
  val_features.append(np.array(item['text']))
  val_labels.append(item['hyperpartisan'])
val_features = np.array(val_features)
len(val_features)

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm

In [ ]:
train_data = TensorDataset(torch.from_numpy(train_features), torch.tensor(train_labels))
val_data = TensorDataset(torch.from_numpy(val_features), torch.tensor(val_labels))

In [ ]:
torch.save(train_data, 'train_data_byarticle.dataset')
torch.save(val_data, 'val_data_bypublisher.dataset')